# Lesson 4

In [5]:
# !pip install python-dotenv

In [7]:
import json
from web3 import Web3

# In the video, we forget to `install_solc`
# from solcx import compile_standard
from solcx import compile_standard, install_solc
import os
from dotenv import load_dotenv
from pathlib import Path

In [6]:
load_dotenv()

True

## Constants

In [17]:
FILENAME = "simple_storage.sol"
SOLC_VERSION = "0.8.0"

## Openup file

In [13]:
simple_storage_file = Path(FILENAME).read_text()

## Install solc

In [18]:
install_solc(SOLC_VERSION)

Version('0.8.0')

## Compile and save

In [25]:
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {FILENAME: {"content": simple_storage_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version=SOLC_VERSION,
)

Save to file

In [26]:
with open("compiled_code.json", "w") as f:
    f.write(json.dumps(compiled_sol, indent = 2))

## get bytecode

In [28]:
bytecode = compiled_sol["contracts"][FILENAME]["SimpleStorage"]["evm"][
    "bytecode"
]["object"]

In [32]:
meta_data = compiled_sol["contracts"][FILENAME]["SimpleStorage"]["metadata"]

## Get ABI

In [33]:
abi = json.loads(meta_data)["output"]["abi"]

Run
```shell
ganache-cli
```

In [61]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
# default chain_id
chain_id = 1337
# from ganache-cli
my_address = "0xF448209280954bAFaf5050ad3F5ac80b1846C262"
private_key = "0x7e9098b2076c5772bee1215206513ba37f3838369f9a2fa8a06067d9e87b96dd"

## Create the contract in Python

In [62]:
SimpleStorage = w3.eth.contract(abi=abi, bytecode=bytecode)

## Get nonce

In [63]:
nonce = w3.eth.getTransactionCount(my_address)

In [64]:
nonce

1

## Submit the transaction that deploys the contract

In [66]:
transaction = SimpleStorage.constructor().buildTransaction({
    "chainId":chain_id,
    "gasPrice": w3.eth.gas_price,
    "from": my_address,
    "nonce": nonce,
})

Sign the transaction

In [67]:
signed_txn = w3.eth.account.sign_transaction(transaction, private_key=private_key)

Send transaction

In [68]:
tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)

In [69]:
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

The contract will be deployed to such address

In [70]:
tx_receipt.contractAddress

'0xe4630B2E36cAE49aC4a5B5f8641B97aF8D98F38e'

## Working with deployed contract

In [71]:
simple_storage = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)

In [72]:
greeting_transaction = simple_storage.functions.store(15).buildTransaction(
    {
        "chainId": chain_id,
        "gasPrice": w3.eth.gas_price,
        "from": my_address,
        "nonce": nonce + 1,
    }
)

In [74]:
signed_greeting_txn = w3.eth.account.sign_transaction(
    greeting_transaction, private_key=private_key
)

In [75]:
tx_greeting_hash = w3.eth.send_raw_transaction(signed_greeting_txn.rawTransaction)

In [76]:
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_greeting_hash)